In [ ]:
!git clone https://github.com/keras-team/keras-tuner

In [ ]:
cd keras-tuner

In [ ]:
!pip install .

In [ ]:
# Train a DNN model for prediction

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kerastuner as kt

# upload the data base at the certain path
data = pd.read_csv('/content/trainset.csv')  
x_train = data.loc[0:506, ['A','B','C','D','E','F']]
x_train=np.array(x_train)
y_train = data.loc[0:506, ['target']]
y_train=np.array(y_train)

#Define model
def model_builder(hp):
  model = keras.Sequential()
#Set the input layer
  model.add(keras.layers.Flatten(input_shape=(6,1)))
#Set dropout rate search space
  drop_rate = hp.Choice('drop_rate', 
                            [0.0, 0.1, 0.2, 0.3, 0.4,])
#Set activation function search space
  activation = hp.Choice('activation', 
                            ['relu', 'tanh', 'sigmoid'])
#In here, we tuner the number of layers using for loop
  for i in range(hp.Int('num_layers', 2 , 5)):
    model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                      min_value=6,
                      max_value=50,
                      step=1),
                activation= activation))
  model.add(keras.layers.Dropout(rate=drop_rate))
  model.add(keras.layers.Dense(1, activation='linear'))
#  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
  model.compile(
          optimizer= optimizer,
          loss='mean_absolute_error',
          metrics=['mean_absolute_error'])

#          optimizer=tf.keras.optimizers.optimizer(lr=hp_learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08),

  return model
tuner = kt.RandomSearch(model_builder,
                     objective='val_mean_absolute_error',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='3')
# represent the search space
tuner.search_space_summary()

# search the best hyperparameters
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train,y_train,epochs=50,validation_split=0.2,callbacks=[stop_early])

# recall the overall hyperparameters
tuner.results_summary()

# train model with the best hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)


history=model.fit(x_train, y_train, batch_size=32, epochs=500, validation_split=0.2, validation_freq=1)


# plot the figure illustrating the training loss and validation loss
epochs = len(history.history['loss'])
plt.plot(range(epochs), history.history['loss'], label='loss')
plt.plot(range(epochs), history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
x = np.array([0.0063,2.3100,0.5380,6.5750,65.2000,4.0900])
x=x.reshape(1,6)
y=model(x)
print(y)

#manifest the model structure
model.summary()

#extract the best hyperparameters 
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
!pip3 install pygad

In [ ]:
# Optimization for input features

import numpy as np
import pygad

#initialization
function_inputs = [1,1,1,1,1,1]

#define fitness function
def fitness_func(solution, solution_idx):
    # solution[i] represeent the value of the (i+1)th parameter in solution
    #solution=np.empty(shape=(1,6))
    #solution= np.array().shape(6,1)
    output=model(np.array(solution*function_inputs).reshape(1,6)) # model is built in keras.tuner
    fitness =((1/output)).numpy()[0]
    return fitness

# Number of generations.
num_generations = 50 
# Number of solutions to be selected as parents in the mating pool.
num_parents_mating = 5 
# Number of solutions in the population.
sol_per_pop = 10 
  
# Type of parent selection.
parent_selection_type = "sss"
# Type of the crossover operator.
crossover_type = "single_point" 
# Type of the mutation operator.
mutation_type = "random"

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(
     #Number of generations(iterations).
     num_generations=num_generations,
     num_parents_mating=num_parents_mating,
     fitness_func=fitness_func,
     sol_per_pop=sol_per_pop,
     #the Number of parameters(the number of genes)
     num_genes=6,
     gene_space=[np.arange(0,10,0.01).tolist(),np.arange(0,15,0.01).tolist(),np.arange(0,21,0.01).tolist(),np.arange(0,2,0.01).tolist(),
                 np.arange(0,100,0.01).tolist(),np.arange(0,2,0.01).tolist()],
     
     parent_selection_type=parent_selection_type,
     keep_parents=1,
     crossover_type=crossover_type,
     mutation_percent_genes=50,
     mutation_type=mutation_type,
     callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
ga_instance.plot_result()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=1/solution_fitness))